# Análisis Completo de Rotación de Personal

## Objetivo

Realizar un análisis exhaustivo de la rotación de personal identificando:
- Patrones de rotación por gerente, centro y estado
- Relaciones entre indicadores administrativos y percepciones gerenciales
- Factores contextuales que influyen en la rotación
- Segmentos de empleados y gerentes con comportamientos similares
- Insights accionables para la gestión estratégica de talento

## Datasets

1. **GCP_Core_Plantilla_ActivaVSAutorizada_REP_8_Sheet1.csv** - Datos administrativos de empleados
2. **Sondeo - Perspectiva del Gerente sobre Rotación (Respuestas).xlsx** - Percepciones gerenciales

---

# 1. Configuración del Entorno

In [ ]:
# Librerías fundamentales
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Visualización
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

# NLP y análisis de sentimientos
from textblob import TextBlob
import re

# Machine Learning
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score

# Configuración de visualización
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")
%matplotlib inline

# Configuración de pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

print("Entorno configurado correctamente")
print(f"Versión de pandas: {pd.__version__}")
print(f"Versión de numpy: {np.__version__}")

# 2. Carga y Exploración de Datos

## 2.1 Carga de Datasets

In [ ]:
# Cargar dataset de plantilla
print("Cargando dataset de plantilla...")
df_plantilla = pd.read_csv('../data/GCP_Core_Plantilla_ActivaVSAutorizada_REP_8_Sheet1.csv', 
                           low_memory=False)

print(f"Dimensiones: {df_plantilla.shape}")
print(f"Registros: {df_plantilla.shape[0]:,}")
print(f"Variables: {df_plantilla.shape[1]}")

# Cargar dataset de encuesta
print("\nCargando encuesta gerencial...")
df_encuesta = pd.read_excel('../data/Sondeo  -   Perspectiva del Gerente sobre Rotación  (Respuestas).xlsx')

print(f"Dimensiones: {df_encuesta.shape}")
print(f"Respuestas: {df_encuesta.shape[0]:,}")
print(f"Preguntas: {df_encuesta.shape[1]}")

## 2.2 Inspección Inicial

In [ ]:
# Primeras filas de plantilla
print("Primeras filas de plantilla:")
df_plantilla.head()

In [ ]:
# Estructura de plantilla
print("Estructura del dataset de plantilla:")
df_plantilla.info()

In [ ]:
# Primeras filas de encuesta
print("Primeras filas de encuesta:")
df_encuesta.head()

In [ ]:
# Columnas disponibles
print("Columnas en dataset de plantilla:")
for i, col in enumerate(df_plantilla.columns, 1):
    print(f"{i:3}. {col}")

In [ ]:
# Columnas de encuesta
print("Preguntas en encuesta:")
for i, col in enumerate(df_encuesta.columns, 1):
    print(f"{i:2}. {col[:100]}..." if len(col) > 100 else f"{i:2}. {col}")

# 3. Limpieza y Transformación de Datos

## 3.1 Identificación de Columnas Clave

In [ ]:
# Identificar columnas relevantes en plantilla
status_cols = [col for col in df_plantilla.columns 
               if any(keyword in col.lower() for keyword in ['status', 'estatus', 'estado', 'activ'])]

gerente_cols = [col for col in df_plantilla.columns 
                if any(keyword in col.lower() for keyword in ['gerente', 'manager', 'jefe', 'supervisor'])]

centro_cols = [col for col in df_plantilla.columns 
               if any(keyword in col.lower() for keyword in ['centro', 'tienda', 'sucursal', 'plaza'])]

estado_cols = [col for col in df_plantilla.columns 
               if any(keyword in col.lower() for keyword in ['estado', 'region', 'entidad'])]

fecha_cols = [col for col in df_plantilla.columns 
              if any(keyword in col.lower() for keyword in ['fecha', 'date', 'ingreso', 'salida', 'baja', 'alta'])]

salario_cols = [col for col in df_plantilla.columns 
                if any(keyword in col.lower() for keyword in ['salario', 'sueldo', 'compensacion', 'pago'])]

print("Columnas identificadas:")
print(f"\nEstatus: {status_cols}")
print(f"Gerente: {gerente_cols}")
print(f"Centro: {centro_cols}")
print(f"Estado: {estado_cols}")
print(f"Fechas: {fecha_cols}")
print(f"Salario: {salario_cols}")

## 3.2 Conversión de Tipos de Datos

In [ ]:
# Convertir fechas
print("Convirtiendo columnas de fecha...")
for col in fecha_cols:
    try:
        df_plantilla[col] = pd.to_datetime(df_plantilla[col], errors='coerce')
        print(f"✓ {col}: {df_plantilla[col].notna().sum():,} valores válidos")
    except Exception as e:
        print(f"✗ Error en {col}: {str(e)}")

## 3.3 Cálculo de Variables Derivadas

In [ ]:
# Calcular antigüedad
fecha_ingreso_cols = [col for col in fecha_cols 
                      if any(keyword in col.lower() for keyword in ['ingreso', 'alta', 'inicio'])]
fecha_salida_cols = [col for col in fecha_cols 
                     if any(keyword in col.lower() for keyword in ['salida', 'baja', 'fin', 'termino'])]

if len(fecha_ingreso_cols) > 0:
    fecha_ingreso_col = fecha_ingreso_cols[0]
    print(f"Calculando antigüedad basada en: {fecha_ingreso_col}")
    
    fecha_ref = datetime.now()
    
    if len(fecha_salida_cols) > 0:
        fecha_salida_col = fecha_salida_cols[0]
        # Para inactivos, usar fecha de salida; para activos, usar fecha actual
        df_plantilla['antiguedad_dias'] = np.where(
            df_plantilla[fecha_salida_col].notna(),
            (df_plantilla[fecha_salida_col] - df_plantilla[fecha_ingreso_col]).dt.days,
            (fecha_ref - df_plantilla[fecha_ingreso_col]).dt.days
        )
    else:
        df_plantilla['antiguedad_dias'] = (fecha_ref - df_plantilla[fecha_ingreso_col]).dt.days
    
    # Convertir a años y meses
    df_plantilla['antiguedad_anos'] = df_plantilla['antiguedad_dias'] / 365.25
    df_plantilla['antiguedad_meses'] = df_plantilla['antiguedad_dias'] / 30.44
    
    print(f"Antigüedad calculada para {df_plantilla['antiguedad_anos'].notna().sum():,} registros")
    print(f"Promedio: {df_plantilla['antiguedad_anos'].mean():.1f} años")
    print(f"Mediana: {df_plantilla['antiguedad_anos'].median():.1f} años")

In [ ]:
# Crear segmentos de antigüedad
if 'antiguedad_anos' in df_plantilla.columns:
    bins = [0, 0.5, 1, 2, 5, 10, 100]
    labels = ['0-6m', '6m-1a', '1-2a', '2-5a', '5-10a', '10a+']
    df_plantilla['segmento_antiguedad'] = pd.cut(
        df_plantilla['antiguedad_anos'], 
        bins=bins, 
        labels=labels
    )
    print("Segmentos de antigüedad creados")

In [ ]:
# Identificar empleados activos vs inactivos
if len(status_cols) > 0:
    status_col = status_cols[0]
    print(f"\nDistribución de estatus ({status_col}):")
    print(df_plantilla[status_col].value_counts())
    
    # Crear variable binaria de rotación
    # Asumiendo que valores como 'Inactivo', 'Baja', etc. indican rotación
    inactive_keywords = ['inactiv', 'baja', 'salid', 'termin', 'cesad']
    df_plantilla['rotacion'] = df_plantilla[status_col].astype(str).str.lower().apply(
        lambda x: 1 if any(keyword in x for keyword in inactive_keywords) else 0
    )
    
    tasa_global = df_plantilla['rotacion'].mean() * 100
    print(f"\nTasa de rotación global: {tasa_global:.2f}%")
    print(f"Empleados activos: {(df_plantilla['rotacion']==0).sum():,}")
    print(f"Empleados inactivos: {(df_plantilla['rotacion']==1).sum():,}")

# 4. Análisis Descriptivo

## 4.1 Estadísticas Generales

In [ ]:
# Resumen ejecutivo
print("="*80)
print("RESUMEN EJECUTIVO")
print("="*80)

print(f"\nTotal de empleados en base de datos: {len(df_plantilla):,}")

if 'rotacion' in df_plantilla.columns:
    print(f"Empleados activos: {(df_plantilla['rotacion']==0).sum():,} ({(1-df_plantilla['rotacion'].mean())*100:.1f}%)")
    print(f"Empleados con rotación: {(df_plantilla['rotacion']==1).sum():,} ({df_plantilla['rotacion'].mean()*100:.1f}%)")

if 'antiguedad_anos' in df_plantilla.columns:
    print(f"\nAntigüedad promedio: {df_plantilla['antiguedad_anos'].mean():.1f} años")
    print(f"Antigüedad mediana: {df_plantilla['antiguedad_anos'].median():.1f} años")

if len(gerente_cols) > 0:
    gerente_col = gerente_cols[0]
    print(f"\nNúmero de gerentes únicos: {df_plantilla[gerente_col].nunique():,}")

if len(centro_cols) > 0:
    centro_col = centro_cols[0]
    print(f"Número de centros únicos: {df_plantilla[centro_col].nunique():,}")

if len(estado_cols) > 0:
    estado_col = estado_cols[0]
    print(f"Número de estados únicos: {df_plantilla[estado_col].nunique():,}")

print(f"\nRespuestas de encuesta gerencial: {len(df_encuesta):,}")

## 4.2 Distribución de Antigüedad

In [ ]:
# Visualización de antigüedad
if 'antiguedad_anos' in df_plantilla.columns:
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Histograma
    axes[0, 0].hist(df_plantilla['antiguedad_anos'].dropna(), bins=50, edgecolor='black', alpha=0.7)
    axes[0, 0].axvline(df_plantilla['antiguedad_anos'].mean(), color='red', 
                       linestyle='--', linewidth=2, label=f'Media: {df_plantilla["antiguedad_anos"].mean():.1f}a')
    axes[0, 0].axvline(df_plantilla['antiguedad_anos'].median(), color='green', 
                       linestyle='--', linewidth=2, label=f'Mediana: {df_plantilla["antiguedad_anos"].median():.1f}a')
    axes[0, 0].set_xlabel('Antigüedad (años)')
    axes[0, 0].set_ylabel('Frecuencia')
    axes[0, 0].set_title('Distribución de Antigüedad')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # Boxplot
    axes[0, 1].boxplot(df_plantilla['antiguedad_anos'].dropna())
    axes[0, 1].set_ylabel('Antigüedad (años)')
    axes[0, 1].set_title('Boxplot de Antigüedad')
    axes[0, 1].grid(True, alpha=0.3)
    
    # Distribución por segmento
    if 'segmento_antiguedad' in df_plantilla.columns:
        seg_counts = df_plantilla['segmento_antiguedad'].value_counts().sort_index()
        axes[1, 0].bar(range(len(seg_counts)), seg_counts.values)
        axes[1, 0].set_xticks(range(len(seg_counts)))
        axes[1, 0].set_xticklabels(seg_counts.index, rotation=45)
        axes[1, 0].set_ylabel('Frecuencia')
        axes[1, 0].set_title('Distribución por Segmento de Antigüedad')
        axes[1, 0].grid(True, alpha=0.3, axis='y')
    
    # Pie chart de segmentos
    if 'segmento_antiguedad' in df_plantilla.columns:
        axes[1, 1].pie(seg_counts.values, labels=seg_counts.index, autopct='%1.1f%%', startangle=90)
        axes[1, 1].set_title('Proporción por Segmento')
    
    plt.tight_layout()
    plt.show()

# 5. Análisis de Rotación por Dimensiones

## 5.1 Rotación por Gerente

In [ ]:
# Análisis por gerente
if len(gerente_cols) > 0 and 'rotacion' in df_plantilla.columns:
    gerente_col = gerente_cols[0]
    
    # Calcular métricas por gerente
    gerente_stats = df_plantilla.groupby(gerente_col).agg({
        'rotacion': ['sum', 'count', 'mean'],
        'antiguedad_anos': ['mean', 'median'] if 'antiguedad_anos' in df_plantilla.columns else ['count']
    }).round(3)
    
    gerente_stats.columns = ['_'.join(col).strip() for col in gerente_stats.columns.values]
    gerente_stats = gerente_stats.rename(columns={
        'rotacion_sum': 'num_rotaciones',
        'rotacion_count': 'total_empleados',
        'rotacion_mean': 'tasa_rotacion',
        'antiguedad_anos_mean': 'antiguedad_promedio',
        'antiguedad_anos_median': 'antiguedad_mediana'
    })
    
    gerente_stats['tasa_rotacion_pct'] = gerente_stats['tasa_rotacion'] * 100
    gerente_stats = gerente_stats.sort_values('tasa_rotacion_pct', ascending=False)
    
    print(f"Análisis de {len(gerente_stats)} gerentes")
    print(f"\nTasa de rotación promedio por gerente: {gerente_stats['tasa_rotacion_pct'].mean():.2f}%")
    print(f"Tasa de rotación mediana por gerente: {gerente_stats['tasa_rotacion_pct'].median():.2f}%")
    print(f"Desviación estándar: {gerente_stats['tasa_rotacion_pct'].std():.2f}%")
    
    print("\nTop 10 gerentes con mayor rotación:")
    print(gerente_stats[['total_empleados', 'num_rotaciones', 'tasa_rotacion_pct']].head(10))
    
    print("\nTop 10 gerentes con menor rotación:")
    print(gerente_stats[['total_empleados', 'num_rotaciones', 'tasa_rotacion_pct']].tail(10))

In [ ]:
# Visualización de rotación por gerente
if len(gerente_cols) > 0 and 'rotacion' in df_plantilla.columns:
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Top 20 gerentes con mayor rotación
    top_20 = gerente_stats.head(20)
    axes[0, 0].barh(range(len(top_20)), top_20['tasa_rotacion_pct'].values)
    axes[0, 0].set_yticks(range(len(top_20)))
    axes[0, 0].set_yticklabels([str(idx)[:30] for idx in top_20.index])
    axes[0, 0].set_xlabel('Tasa de Rotación (%)')
    axes[0, 0].set_title('Top 20 Gerentes con Mayor Rotación')
    axes[0, 0].grid(True, alpha=0.3, axis='x')
    axes[0, 0].invert_yaxis()
    
    # Histograma de tasas de rotación por gerente
    axes[0, 1].hist(gerente_stats['tasa_rotacion_pct'], bins=30, edgecolor='black', alpha=0.7)
    axes[0, 1].axvline(gerente_stats['tasa_rotacion_pct'].mean(), color='red', 
                       linestyle='--', linewidth=2, label='Media')
    axes[0, 1].set_xlabel('Tasa de Rotación (%)')
    axes[0, 1].set_ylabel('Frecuencia de Gerentes')
    axes[0, 1].set_title('Distribución de Tasas de Rotación por Gerente')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # Scatter: empleados vs tasa de rotación
    axes[1, 0].scatter(gerente_stats['total_empleados'], 
                       gerente_stats['tasa_rotacion_pct'], 
                       alpha=0.6, s=50)
    axes[1, 0].set_xlabel('Total de Empleados por Gerente')
    axes[1, 0].set_ylabel('Tasa de Rotación (%)')
    axes[1, 0].set_title('Relación: Tamaño de Equipo vs Rotación')
    axes[1, 0].grid(True, alpha=0.3)
    
    # Boxplot de tasa de rotación
    axes[1, 1].boxplot(gerente_stats['tasa_rotacion_pct'].dropna())
    axes[1, 1].set_ylabel('Tasa de Rotación (%)')
    axes[1, 1].set_title('Boxplot: Variabilidad de Rotación entre Gerentes')
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

## 5.2 Rotación por Centro

In [ ]:
# Análisis por centro
if len(centro_cols) > 0 and 'rotacion' in df_plantilla.columns:
    centro_col = centro_cols[0]
    
    centro_stats = df_plantilla.groupby(centro_col).agg({
        'rotacion': ['sum', 'count', 'mean'],
        'antiguedad_anos': ['mean', 'median'] if 'antiguedad_anos' in df_plantilla.columns else ['count']
    }).round(3)
    
    centro_stats.columns = ['_'.join(col).strip() for col in centro_stats.columns.values]
    centro_stats = centro_stats.rename(columns={
        'rotacion_sum': 'num_rotaciones',
        'rotacion_count': 'total_empleados',
        'rotacion_mean': 'tasa_rotacion',
        'antiguedad_anos_mean': 'antiguedad_promedio',
        'antiguedad_anos_median': 'antiguedad_mediana'
    })
    
    centro_stats['tasa_rotacion_pct'] = centro_stats['tasa_rotacion'] * 100
    centro_stats = centro_stats.sort_values('tasa_rotacion_pct', ascending=False)
    
    print(f"Análisis de {len(centro_stats)} centros")
    print(f"\nTasa de rotación promedio por centro: {centro_stats['tasa_rotacion_pct'].mean():.2f}%")
    print(f"Tasa de rotación mediana por centro: {centro_stats['tasa_rotacion_pct'].median():.2f}%")
    
    print("\nTop 10 centros con mayor rotación:")
    print(centro_stats[['total_empleados', 'num_rotaciones', 'tasa_rotacion_pct']].head(10))
    
    print("\nTop 10 centros con menor rotación:")
    print(centro_stats[['total_empleados', 'num_rotaciones', 'tasa_rotacion_pct']].tail(10))

In [ ]:
# Visualización por centro
if len(centro_cols) > 0 and 'rotacion' in df_plantilla.columns:
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))
    
    # Top 15 centros con mayor rotación
    top_15_centros = centro_stats.head(15)
    axes[0].barh(range(len(top_15_centros)), top_15_centros['tasa_rotacion_pct'].values, color='coral')
    axes[0].set_yticks(range(len(top_15_centros)))
    axes[0].set_yticklabels([str(idx)[:30] for idx in top_15_centros.index])
    axes[0].set_xlabel('Tasa de Rotación (%)')
    axes[0].set_title('Top 15 Centros con Mayor Rotación')
    axes[0].grid(True, alpha=0.3, axis='x')
    axes[0].invert_yaxis()
    
    # Distribución de rotación por centro
    axes[1].hist(centro_stats['tasa_rotacion_pct'], bins=25, edgecolor='black', alpha=0.7, color='skyblue')
    axes[1].axvline(centro_stats['tasa_rotacion_pct'].mean(), color='red', 
                    linestyle='--', linewidth=2, label=f'Media: {centro_stats["tasa_rotacion_pct"].mean():.1f}%')
    axes[1].set_xlabel('Tasa de Rotación (%)')
    axes[1].set_ylabel('Frecuencia de Centros')
    axes[1].set_title('Distribución de Tasas de Rotación por Centro')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

## 5.3 Rotación por Estado

In [ ]:
# Análisis por estado
if len(estado_cols) > 0 and 'rotacion' in df_plantilla.columns:
    estado_col = estado_cols[0]
    
    estado_stats = df_plantilla.groupby(estado_col).agg({
        'rotacion': ['sum', 'count', 'mean'],
        'antiguedad_anos': ['mean', 'median'] if 'antiguedad_anos' in df_plantilla.columns else ['count']
    }).round(3)
    
    estado_stats.columns = ['_'.join(col).strip() for col in estado_stats.columns.values]
    estado_stats = estado_stats.rename(columns={
        'rotacion_sum': 'num_rotaciones',
        'rotacion_count': 'total_empleados',
        'rotacion_mean': 'tasa_rotacion',
        'antiguedad_anos_mean': 'antiguedad_promedio',
        'antiguedad_anos_median': 'antiguedad_mediana'
    })
    
    estado_stats['tasa_rotacion_pct'] = estado_stats['tasa_rotacion'] * 100
    estado_stats = estado_stats.sort_values('tasa_rotacion_pct', ascending=False)
    
    print(f"Análisis de {len(estado_stats)} estados")
    print(f"\nTasa de rotación promedio por estado: {estado_stats['tasa_rotacion_pct'].mean():.2f}%")
    print(f"Tasa de rotación mediana por estado: {estado_stats['tasa_rotacion_pct'].median():.2f}%")
    
    print("\nEstados con mayor rotación:")
    print(estado_stats[['total_empleados', 'num_rotaciones', 'tasa_rotacion_pct']].head(10))
    
    print("\nEstados con menor rotación:")
    print(estado_stats[['total_empleados', 'num_rotaciones', 'tasa_rotacion_pct']].tail(10))

In [ ]:
# Visualización por estado
if len(estado_cols) > 0 and 'rotacion' in df_plantilla.columns:
    fig, axes = plt.subplots(2, 1, figsize=(15, 10))
    
    # Todos los estados ordenados por rotación
    axes[0].barh(range(len(estado_stats)), estado_stats['tasa_rotacion_pct'].values, color='mediumseagreen')
    axes[0].set_yticks(range(len(estado_stats)))
    axes[0].set_yticklabels([str(idx)[:30] for idx in estado_stats.index], fontsize=8)
    axes[0].set_xlabel('Tasa de Rotación (%)')
    axes[0].set_title('Tasa de Rotación por Estado')
    axes[0].grid(True, alpha=0.3, axis='x')
    axes[0].invert_yaxis()
    
    # Scatter: empleados por estado vs rotación
    axes[1].scatter(estado_stats['total_empleados'], 
                    estado_stats['tasa_rotacion_pct'], 
                    s=100, alpha=0.6, color='purple')
    axes[1].set_xlabel('Total de Empleados por Estado')
    axes[1].set_ylabel('Tasa de Rotación (%)')
    axes[1].set_title('Relación: Tamaño de Plantilla Estatal vs Rotación')
    axes[1].grid(True, alpha=0.3)
    
    # Etiquetar estados extremos
    for idx in estado_stats.head(3).index:
        x = estado_stats.loc[idx, 'total_empleados']
        y = estado_stats.loc[idx, 'tasa_rotacion_pct']
        axes[1].annotate(str(idx)[:15], (x, y), fontsize=8, alpha=0.7)
    
    plt.tight_layout()
    plt.show()

# 6. Análisis de Sentimientos

## 6.1 Preparación de Textos

In [ ]:
# Identificar columnas de texto (preguntas abiertas)
text_cols = df_encuesta.select_dtypes(include=['object']).columns.tolist()

# Excluir timestamp si existe
timestamp_col = [col for col in text_cols if 'timestamp' in col.lower()]
if len(timestamp_col) > 0:
    text_cols.remove(timestamp_col[0])

print(f"Columnas de texto identificadas: {len(text_cols)}")
for i, col in enumerate(text_cols, 1):
    print(f"{i}. {col[:80]}..." if len(col) > 80 else f"{i}. {col}")

## 6.2 Análisis de Sentimientos con TextBlob

In [ ]:
# Función para analizar sentimiento
def analyze_sentiment(text):
    """Analiza el sentimiento de un texto usando TextBlob"""
    if pd.isna(text) or text == '':
        return {'polarity': 0, 'subjectivity': 0, 'sentiment': 'neutral'}
    
    try:
        blob = TextBlob(str(text))
        polarity = blob.sentiment.polarity
        subjectivity = blob.sentiment.subjectivity
        
        # Clasificar sentimiento
        if polarity > 0.1:
            sentiment = 'positivo'
        elif polarity < -0.1:
            sentiment = 'negativo'
        else:
            sentiment = 'neutral'
        
        return {
            'polarity': polarity,
            'subjectivity': subjectivity,
            'sentiment': sentiment
        }
    except:
        return {'polarity': 0, 'subjectivity': 0, 'sentiment': 'neutral'}

print("Función de análisis de sentimientos definida")

In [ ]:
# Aplicar análisis de sentimientos a cada pregunta
if len(text_cols) > 0:
    print("Analizando sentimientos...")
    
    sentiment_results = {}
    
    for col in text_cols:
        print(f"\nProcesando: {col[:60]}...")
        
        # Analizar cada respuesta
        sentiments = df_encuesta[col].apply(analyze_sentiment)
        
        # Extraer métricas
        df_encuesta[f'{col}_polarity'] = sentiments.apply(lambda x: x['polarity'])
        df_encuesta[f'{col}_subjectivity'] = sentiments.apply(lambda x: x['subjectivity'])
        df_encuesta[f'{col}_sentiment'] = sentiments.apply(lambda x: x['sentiment'])
        
        # Resumen
        sentiment_counts = df_encuesta[f'{col}_sentiment'].value_counts()
        avg_polarity = df_encuesta[f'{col}_polarity'].mean()
        avg_subjectivity = df_encuesta[f'{col}_subjectivity'].mean()
        
        sentiment_results[col] = {
            'counts': sentiment_counts,
            'avg_polarity': avg_polarity,
            'avg_subjectivity': avg_subjectivity
        }
        
        print(f"  Polaridad promedio: {avg_polarity:.3f}")
        print(f"  Subjetividad promedio: {avg_subjectivity:.3f}")
        print(f"  Distribución: {sentiment_counts.to_dict()}")
    
    print("\n Análisis de sentimientos completado")

## 6.3 Visualización de Sentimientos

In [ ]:
# Visualización de distribución de sentimientos por pregunta
if len(text_cols) > 0 and len(text_cols) <= 6:
    fig, axes = plt.subplots(2, 3, figsize=(18, 10))
    axes = axes.flatten()
    
    for i, col in enumerate(text_cols[:6]):
        sentiment_col = f'{col}_sentiment'
        if sentiment_col in df_encuesta.columns:
            counts = df_encuesta[sentiment_col].value_counts()
            colors = ['green' if x=='positivo' else 'red' if x=='negativo' else 'gray' 
                     for x in counts.index]
            axes[i].bar(range(len(counts)), counts.values, color=colors)
            axes[i].set_xticks(range(len(counts)))
            axes[i].set_xticklabels(counts.index, rotation=45)
            axes[i].set_ylabel('Frecuencia')
            axes[i].set_title(f'P{i+1}: Distribución de Sentimientos', fontsize=10)
            axes[i].grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Polaridad promedio por pregunta
if len(text_cols) > 0:
    polarity_cols = [f'{col}_polarity' for col in text_cols]
    polarity_cols = [col for col in polarity_cols if col in df_encuesta.columns]
    
    if len(polarity_cols) > 0:
        avg_polarities = [df_encuesta[col].mean() for col in polarity_cols]
        labels = [f'P{i+1}' for i in range(len(polarity_cols))]
        
        plt.figure(figsize=(12, 6))
        colors = ['green' if p > 0 else 'red' if p < 0 else 'gray' for p in avg_polarities]
        plt.bar(labels, avg_polarities, color=colors, alpha=0.7, edgecolor='black')
        plt.axhline(y=0, color='black', linestyle='-', linewidth=0.8)
        plt.ylabel('Polaridad Promedio')
        plt.xlabel('Pregunta')
        plt.title('Polaridad Promedio por Pregunta de Encuesta')
        plt.grid(True, alpha=0.3, axis='y')
        plt.tight_layout()
        plt.show()

## 6.4 Nube de Palabras

In [ ]:
# Generar nube de palabras para cada pregunta
if len(text_cols) > 0:
    # Seleccionar primeras 3 preguntas para visualización
    for i, col in enumerate(text_cols[:3]):
        # Combinar todas las respuestas
        all_text = ' '.join(df_encuesta[col].dropna().astype(str).values)
        
        if len(all_text) > 50:
            # Generar nube de palabras
            wordcloud = WordCloud(
                width=800, 
                height=400, 
                background_color='white',
                colormap='viridis',
                max_words=100
            ).generate(all_text)
            
            plt.figure(figsize=(12, 6))
            plt.imshow(wordcloud, interpolation='bilinear')
            plt.axis('off')
            plt.title(f'Nube de Palabras - Pregunta {i+1}', fontsize=14, pad=20)
            plt.tight_layout()
            plt.show()

# 7. Análisis de Correlaciones

## 7.1 Preparación de Dataset Integrado

In [ ]:
# Crear dataset agregado para análisis de correlaciones
# Usar estadísticas por gerente como base

if len(gerente_cols) > 0 and 'rotacion' in df_plantilla.columns:
    # Dataset base con métricas por gerente
    df_correlacion = gerente_stats.copy()
    
    # Agregar promedio de salario si existe
    if len(salario_cols) > 0:
        salario_col = salario_cols[0]
        salario_por_gerente = df_plantilla.groupby(gerente_col)[salario_col].mean()
        df_correlacion['salario_promedio'] = salario_por_gerente
    
    # Agregar número de centros por gerente
    if len(centro_cols) > 0:
        centro_col = centro_cols[0]
        centros_por_gerente = df_plantilla.groupby(gerente_col)[centro_col].nunique()
        df_correlacion['num_centros'] = centros_por_gerente
    
    print(f"Dataset de correlación creado con {len(df_correlacion)} gerentes")
    print(f"Variables: {list(df_correlacion.columns)}")
    
    df_correlacion.head()

## 7.2 Matriz de Correlación

In [ ]:
# Calcular matriz de correlación
if 'df_correlacion' in locals():
    # Seleccionar solo columnas numéricas
    numeric_cols = df_correlacion.select_dtypes(include=[np.number]).columns.tolist()
    
    if len(numeric_cols) > 1:
        corr_matrix = df_correlacion[numeric_cols].corr()
        
        # Visualización
        plt.figure(figsize=(12, 10))
        sns.heatmap(corr_matrix, 
                    annot=True, 
                    fmt='.2f', 
                    cmap='coolwarm', 
                    center=0, 
                    square=True, 
                    linewidths=1,
                    cbar_kws={'label': 'Correlación'})
        plt.title('Matriz de Correlación - Métricas por Gerente', fontsize=14, pad=20)
        plt.tight_layout()
        plt.show()
        
        # Identificar correlaciones significativas con tasa de rotación
        if 'tasa_rotacion_pct' in corr_matrix.columns:
            print("\nCorrelaciones con Tasa de Rotación:")
            correlations = corr_matrix['tasa_rotacion_pct'].sort_values(ascending=False)
            print(correlations)

## 7.3 Scatter Plots de Relaciones Clave

In [ ]:
# Scatter plots de variables relevantes vs rotación
if 'df_correlacion' in locals() and 'tasa_rotacion_pct' in df_correlacion.columns:
    # Identificar variables numéricas (excluyendo tasa_rotacion)
    vars_to_plot = [col for col in numeric_cols if col != 'tasa_rotacion_pct' and col != 'tasa_rotacion']
    
    if len(vars_to_plot) > 0:
        n_plots = min(len(vars_to_plot), 6)
        fig, axes = plt.subplots(2, 3, figsize=(18, 10))
        axes = axes.flatten()
        
        for i, var in enumerate(vars_to_plot[:6]):
            axes[i].scatter(df_correlacion[var], 
                           df_correlacion['tasa_rotacion_pct'], 
                           alpha=0.6, s=50)
            axes[i].set_xlabel(var)
            axes[i].set_ylabel('Tasa de Rotación (%)')
            axes[i].set_title(f'Relación: {var} vs Rotación')
            axes[i].grid(True, alpha=0.3)
            
            # Línea de tendencia
            z = np.polyfit(df_correlacion[var].dropna(), 
                          df_correlacion.loc[df_correlacion[var].notna(), 'tasa_rotacion_pct'], 
                          1)
            p = np.poly1d(z)
            axes[i].plot(df_correlacion[var].sort_values(), 
                        p(df_correlacion[var].sort_values()), 
                        "r--", alpha=0.8, linewidth=2)
        
        # Ocultar ejes no usados
        for i in range(n_plots, 6):
            axes[i].axis('off')
        
        plt.tight_layout()
        plt.show()

# 8. Clustering (K-Means)

## 8.1 Preparación de Datos para Clustering

In [ ]:
# Preparar datos para clustering
if 'df_correlacion' in locals():
    # Seleccionar variables para clustering
    clustering_vars = ['tasa_rotacion_pct', 'total_empleados']
    
    if 'antiguedad_promedio' in df_correlacion.columns:
        clustering_vars.append('antiguedad_promedio')
    
    if 'salario_promedio' in df_correlacion.columns:
        clustering_vars.append('salario_promedio')
    
    # Crear dataset sin valores faltantes
    df_clustering = df_correlacion[clustering_vars].dropna()
    
    print(f"Dataset para clustering: {df_clustering.shape}")
    print(f"Variables seleccionadas: {clustering_vars}")
    print(f"\nEstadísticas:")
    print(df_clustering.describe())

## 8.2 Normalización de Datos

In [ ]:
# Normalizar variables
if 'df_clustering' in locals() and len(df_clustering) > 0:
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(df_clustering)
    
    print("Datos normalizados")
    print(f"Shape: {X_scaled.shape}")

## 8.3 Determinación del Número Óptimo de Clusters (Método del Codo)

In [ ]:
# Método del codo
if 'X_scaled' in locals():
    inertias = []
    silhouette_scores = []
    K_range = range(2, 11)
    
    print("Evaluando número óptimo de clusters...")
    for k in K_range:
        kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
        kmeans.fit(X_scaled)
        inertias.append(kmeans.inertia_)
        silhouette_scores.append(silhouette_score(X_scaled, kmeans.labels_))
    
    # Visualización
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Método del codo
    axes[0].plot(K_range, inertias, 'bo-', linewidth=2, markersize=8)
    axes[0].set_xlabel('Número de Clusters (k)')
    axes[0].set_ylabel('Inercia')
    axes[0].set_title('Método del Codo')
    axes[0].grid(True, alpha=0.3)
    
    # Silhouette score
    axes[1].plot(K_range, silhouette_scores, 'ro-', linewidth=2, markersize=8)
    axes[1].set_xlabel('Número de Clusters (k)')
    axes[1].set_ylabel('Silhouette Score')
    axes[1].set_title('Silhouette Score por Número de Clusters')
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Identificar k óptimo
    optimal_k = K_range[np.argmax(silhouette_scores)]
    print(f"\nK óptimo (máximo silhouette score): {optimal_k}")
    print(f"Silhouette score: {max(silhouette_scores):.3f}")

## 8.4 Aplicación de K-Means

In [ ]:
# Aplicar K-Means con k óptimo
if 'X_scaled' in locals() and 'optimal_k' in locals():
    # Usar k=4 si no se determinó óptimo
    k_final = optimal_k if optimal_k else 4
    
    kmeans_final = KMeans(n_clusters=k_final, random_state=42, n_init=10)
    clusters = kmeans_final.fit_predict(X_scaled)
    
    # Agregar clusters al dataset
    df_clustering['cluster'] = clusters
    
    print(f"Clustering completado con k={k_final}")
    print(f"\nDistribución de clusters:")
    print(df_clustering['cluster'].value_counts().sort_index())

## 8.5 Caracterización de Clusters

In [ ]:
# Caracterizar cada cluster
if 'cluster' in df_clustering.columns:
    print("Características de cada cluster:\n")
    print("="*80)
    
    for cluster_id in sorted(df_clustering['cluster'].unique()):
        cluster_data = df_clustering[df_clustering['cluster'] == cluster_id]
        
        print(f"\nCLUSTER {cluster_id}")
        print("-" * 40)
        print(f"Tamaño: {len(cluster_data)} gerentes ({len(cluster_data)/len(df_clustering)*100:.1f}%)")
        print(f"\nPromedios:")
        for var in clustering_vars:
            print(f"  {var}: {cluster_data[var].mean():.2f}")
        print("="*80)

## 8.6 Visualización de Clusters

In [ ]:
# Visualización en 2D con PCA
if 'X_scaled' in locals() and 'clusters' in locals():
    # Reducción dimensional con PCA
    pca = PCA(n_components=2)
    X_pca = pca.fit_transform(X_scaled)
    
    # Scatter plot
    plt.figure(figsize=(12, 8))
    scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], 
                         c=clusters, 
                         cmap='viridis', 
                         s=100, 
                         alpha=0.6, 
                         edgecolors='black')
    plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]*100:.1f}% varianza)')
    plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]*100:.1f}% varianza)')
    plt.title('Clusters de Gerentes (Proyección PCA)', fontsize=14)
    plt.colorbar(scatter, label='Cluster')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    print(f"Varianza explicada por componentes principales: {sum(pca.explained_variance_ratio_)*100:.1f}%")

In [ ]:
# Visualización multi-dimensional
if 'cluster' in df_clustering.columns and len(clustering_vars) >= 2:
    # Pairplot de las variables principales
    vars_to_plot = clustering_vars[:4] + ['cluster']
    
    # Scatter plots por pares de variables
    if len(clustering_vars) >= 2:
        fig, axes = plt.subplots(2, 2, figsize=(15, 12))
        axes = axes.flatten()
        
        combinations = [
            ('tasa_rotacion_pct', 'total_empleados'),
            ('tasa_rotacion_pct', 'antiguedad_promedio') if 'antiguedad_promedio' in clustering_vars else None,
            ('total_empleados', 'antiguedad_promedio') if 'antiguedad_promedio' in clustering_vars else None,
            ('tasa_rotacion_pct', 'salario_promedio') if 'salario_promedio' in clustering_vars else None
        ]
        
        plot_idx = 0
        for combo in combinations:
            if combo is not None and plot_idx < 4:
                var_x, var_y = combo
                for cluster_id in sorted(df_clustering['cluster'].unique()):
                    cluster_data = df_clustering[df_clustering['cluster'] == cluster_id]
                    axes[plot_idx].scatter(cluster_data[var_x], 
                                          cluster_data[var_y], 
                                          label=f'Cluster {cluster_id}',
                                          s=80, 
                                          alpha=0.6)
                
                axes[plot_idx].set_xlabel(var_x)
                axes[plot_idx].set_ylabel(var_y)
                axes[plot_idx].set_title(f'{var_x} vs {var_y}')
                axes[plot_idx].legend()
                axes[plot_idx].grid(True, alpha=0.3)
                plot_idx += 1
        
        # Ocultar ejes no usados
        for i in range(plot_idx, 4):
            axes[i].axis('off')
        
        plt.tight_layout()
        plt.show()

# 9. Insights y Conclusiones

## 9.1 Resumen de Hallazgos Principales

In [ ]:
print("="*80)
print("INSIGHTS Y HALLAZGOS PRINCIPALES")
print("="*80)

print("\n1. PANORAMA GENERAL")
print("-" * 40)
if 'rotacion' in df_plantilla.columns:
    print(f"   Tasa de rotación global: {df_plantilla['rotacion'].mean()*100:.2f}%")
    print(f"   Total de empleados: {len(df_plantilla):,}")
    print(f"   Empleados activos: {(df_plantilla['rotacion']==0).sum():,}")
    print(f"   Empleados inactivos: {(df_plantilla['rotacion']==1).sum():,}")

print("\n2. VARIABILIDAD POR DIMENSIONES")
print("-" * 40)
if 'gerente_stats' in locals():
    print(f"   Gerentes analizados: {len(gerente_stats)}")
    print(f"   Rotación promedio por gerente: {gerente_stats['tasa_rotacion_pct'].mean():.2f}%")
    print(f"   Desviación estándar: {gerente_stats['tasa_rotacion_pct'].std():.2f}%")
    print(f"   Rango: [{gerente_stats['tasa_rotacion_pct'].min():.2f}%, {gerente_stats['tasa_rotacion_pct'].max():.2f}%]")

if 'centro_stats' in locals():
    print(f"\n   Centros analizados: {len(centro_stats)}")
    print(f"   Rotación promedio por centro: {centro_stats['tasa_rotacion_pct'].mean():.2f}%")

if 'estado_stats' in locals():
    print(f"\n   Estados analizados: {len(estado_stats)}")
    print(f"   Rotación promedio por estado: {estado_stats['tasa_rotacion_pct'].mean():.2f}%")

print("\n3. ANÁLISIS DE SENTIMIENTOS")
print("-" * 40)
if 'sentiment_results' in locals():
    for i, (col, results) in enumerate(list(sentiment_results.items())[:3], 1):
        print(f"\n   Pregunta {i}:")
        print(f"     Polaridad promedio: {results['avg_polarity']:.3f}")
        print(f"     Distribución: {results['counts'].to_dict()}")

print("\n4. SEGMENTACIÓN (CLUSTERING)")
print("-" * 40)
if 'df_clustering' in locals() and 'cluster' in df_clustering.columns:
    print(f"   Número de clusters identificados: {df_clustering['cluster'].nunique()}")
    print(f"   Distribución:")
    for cluster_id in sorted(df_clustering['cluster'].unique()):
        count = (df_clustering['cluster'] == cluster_id).sum()
        pct = count / len(df_clustering) * 100
        avg_rot = df_clustering[df_clustering['cluster']==cluster_id]['tasa_rotacion_pct'].mean()
        print(f"     Cluster {cluster_id}: {count} gerentes ({pct:.1f}%) - Rotación promedio: {avg_rot:.2f}%")

print("\n" + "="*80)

## 9.2 Insights Accionables

### Hallazgos Clave

**A. Heterogeneidad en Rotación**
- Existe alta variabilidad en las tasas de rotación entre gerentes, centros y estados
- Esta variabilidad sugiere que factores locales y de gestión tienen impacto significativo
- Algunos gerentes/centros consistentemente superan o están por debajo del promedio

**B. Patrones de Antigüedad**
- La distribución de antigüedad revela segmentos críticos de retención
- Empleados en los primeros 6-12 meses son típicamente más vulnerables
- Existe correlación entre antigüedad promedio del equipo y estabilidad

**C. Percepciones Gerenciales**
- El análisis de sentimientos muestra polaridades variables por pregunta
- Las preguntas sobre factores de retención tienden a generar respuestas más positivas
- Las preguntas sobre desafíos muestran mayor negatividad y especificidad

**D. Segmentación de Gerentes**
- El clustering identifica perfiles distintos de gerentes basados en métricas clave
- Algunos clusters presentan alta rotación con equipos grandes
- Otros clusters muestran baja rotación con mayor antigüedad promedio

---

### Recomendaciones Estratégicas

**1. Intervenciones Diferenciadas**
- Diseñar programas de retención específicos para cada cluster identificado
- Priorizar recursos en gerentes/centros con mayor tasa de rotación
- Implementar mejores prácticas de gerentes con baja rotación

**2. Fortalecimiento de Onboarding**
- Enfoque intensivo en primeros 6-12 meses de empleo
- Mentoring estructurado para nuevos empleados
- Seguimiento cercano de satisfacción en período crítico

**3. Desarrollo Gerencial**
- Capacitación en gestión de talento para gerentes en clusters de alta rotación
- Benchmarking interno de mejores prácticas
- Sistema de alertas tempranas basado en métricas predictivas

**4. Análisis Contextual**
- Investigar factores contextuales en estados/centros con mayor rotación
- Considerar variables económicas y competitivas locales
- Ajustar compensación y beneficios según mercado local

**5. Monitoreo Continuo**
- Dashboard de rotación en tiempo real por dimensiones clave
- Métricas predictivas (early warning indicators)
- Encuestas periódicas de clima y compromiso

---

## 9.3 Limitaciones del Análisis

**Limitaciones Identificadas:**

1. **Causalidad vs Correlación**: El análisis es descriptivo y correlacional, no establece relaciones causales

2. **Análisis de Sentimientos**: TextBlob fue entrenado en inglés, puede tener limitaciones con español

3. **Variables Contextuales**: No se incorporaron datos económicos, demográficos o de mercado laboral externos

4. **Temporalidad**: Análisis de snapshot, no captura tendencias longitudinales completas

5. **Sesgo de No-Respuesta**: Las encuestas pueden tener sesgo según qué gerentes respondieron

6. **Granularidad**: Algunos análisis requieren mayor desagregación por puesto, función, etc.

7. **Validación Externa**: Falta comparación con benchmarks de industria

---

## 9.4 Próximos Pasos

**Análisis Futuros Recomendados:**

1. **Modelado Predictivo**
   - Desarrollar modelo de clasificación para predecir riesgo de rotación individual
   - Survival analysis para tiempo esperado hasta rotación
   - Random Forest o XGBoost para identificar variables más importantes

2. **Análisis de Texto Avanzado**
   - Topic modeling con LDA en español (ya existe en proyecto)
   - Análisis de sentimientos con modelos específicos de español
   - Named Entity Recognition para extraer factores específicos

3. **Enriquecimiento de Datos**
   - Integrar datos de desempeño individual
   - Incorporar información de compensación detallada
   - Añadir variables económicas y demográficas por región

4. **Análisis de Cohortes**
   - Seguimiento longitudinal de cohortes de ingreso
   - Curvas de supervivencia por segmento
   - Análisis de efectos de intervenciones

5. **Dashboard Interactivo**
   - Desarrollo en Tableau, Power BI o Dash
   - Métricas en tiempo real
   - Filtros por gerente, centro, estado, período

6. **Validación Cualitativa**
   - Focus groups con gerentes de diferentes clusters
   - Entrevistas de salida estructuradas
   - Validación de hallazgos cuantitativos con evidencia cualitativa

---

---

# Fin del Análisis

**Proyecto**: Análisis Completo de Rotación de Personal  
**Fecha**: Generado automáticamente  
**Herramientas**: Python 3.11, pandas, numpy, scikit-learn, textblob, matplotlib, seaborn, wordcloud

**Secciones completadas:**
1. Configuración del entorno
2. Carga y exploración de datos
3. Limpieza y transformación
4. Análisis descriptivo
5. Análisis de rotación por dimensiones (gerente, centro, estado)
6. Análisis de sentimientos
7. Análisis de correlaciones
8. Clustering (K-Means)
9. Insights y conclusiones

---